# CIS 5200: Machine Learning
## Homework 3

In [118]:
import os
import sys

# For autograder only, do not modify this cell.
# True for Google Colab, False for autograder
NOTEBOOK = (os.getenv('IS_AUTOGRADER') is None)
if NOTEBOOK:
    print("[INFO, OK] Google Colab.")
else:
    print("[INFO, OK] Autograder.")
    sys.exit()

[INFO, OK] Google Colab.


### Penngrader setup

In [119]:
# %%capture
!pip install penngrader-client

In [120]:
%%writefile config.yaml
grader_api_url: 'https://23whrwph9h.execute-api.us-east-1.amazonaws.com/default/Grader23'
grader_api_key: 'flfkE736fA6Z8GxMDJe2q8Kfk8UDqjsG3GVqOFOa'

Overwriting config.yaml


In [121]:
# packages for homework
import torch
import torch.nn.functional as F
import torch.nn as nn

from sklearn import datasets
from sklearn.model_selection import train_test_split

import matplotlib
%matplotlib inline
import matplotlib.pyplot as plt

### 1. Decision Trees and Bagging

In this problem, we'll implement a simplified version of random forests. We'll be using the `iris` dataset, which has 4 features that are discretized to $0.1$ steps between $0$ and $8$ (i.e. the set of all possible features is $\{0.1, 0.2, \dots, 7.8, 7.9\}$. Thus, all thresholds that we'll need to consider are $\{0.15, 0.25, \dots, 7.75, 7.85\}$.

Your task in this first part is to finish the implementation of decision trees. We've provided a template for some of the decision tree code, following which you'll finish the bagging algorithm to get a random forest.

1. Entropy (2pts): calculate the entropy of a given vector of labels in the `entropy` function. Note that the generalization of entropy to 3 classes is $H = -\sum_{i=1}^3 p_i\log(p_i)$ where $p_i$ is the proportion of examples with label $i$.
2. Find the best split (1pt): finish the `find_split` function by finding the feature index and value that results in the split minimizing entropy.
3. Build the tree (2pts): finish the `expand_node` function by completing the recursive call for building the decision tree.
4. Predict with tree (2pts): implement the `predict_one` function, which makes a prediction for a single example.

Throughout these problems, the way we represent the decision tree is by using python dicts. In particular, a node is a `dict` that can have the following keys:

1. `node['label']` Return the label that we should predict upon reaching this node. node should **only** have a `label` entry if it is a leaf node.
2. `node['left']` points to another node dict that represents this node's left child.
3. `node['right']` points to another node dict that represents this node's right child.
4. `node['split']` is a tuple containing the feature index and value that this node splits left versus right on.

In our implementation, all comparisons will be **greater than** comparisons, and "yes" answers go **left**. In other words, if `node['split'] = (2, 1.25)`, then we expect to find all data remaining at this node with feature 2 value **greater** than 1.25 in `node['left']`, and feature value **less** than 1.25 in `node['right']`.

Tips:
+ If you have NaNs, you may be dividing by zero.


In [122]:
def entropy(y):
    # Calculate the entropy of a given vector of labels in the `entropy` function.
    #
    # y := Tensor(int) of size (m) -- the given vector of labels
    # Return a float that is your calculated entropy

    # Fill in the rest
    _, count = torch.unique(y, return_counts=True)
    prob = count.float() / len(y)
    entropy = -torch.sum(prob * torch.log2(prob))
    return entropy.item()

def find_split(node, X, y, k=4):
    # Find the best split over all possible splits that minimizes entropy.
    #
    # node := Map(string: value) -- the tree represented as a Map, the key will take four
    #   different string: 'label', 'split','left','right' (See implementation below)
    #   'label': a label node with value as the mode of the labels
    #   'split': the best split node with value as a tuple of feature id and threshold
    #   'left','right': the left and right branch with value as the label node
    # X := Tensor of size (m,d) -- Batch of m examples of demension d
    # y := Tensor(int) of size (m) -- the given vectors of labels of the examples
    # k := int -- the number of classes, with default value as 4
    # Return := tuple of (int, float) -- the feature id and threshold of the best split

    m = y.size(0)
    best_H, best_split = 999, None
    features = torch.randint(0, 4,(k,))

    for feature_idx in features:
        for threshold in torch.arange(0.15,7.9,0.1):
            idx = X[:,feature_idx] > threshold

            if idx.sum() == 0 or idx.sum() == idx.size(0):
                continue

            m_left = idx.sum()
            m_right = (~idx).sum()

            H_left = entropy(y[idx])
            H_right = entropy(y[~idx])

            ## ANSWER
            split_H = (m_left/m)*H_left + (m_right/m)*H_right
            ## END ANSWER

            if split_H < best_H or best_split == None:
                best_H, best_split = split_H, (feature_idx, threshold)
    return best_split

def expand_node(node, X, y, max_depth=0, k=4):
    # Completing the recursive call for building the decision tree
    # node := Map(string: value) -- the tree represented as a Map, the key will take four
    #   different string: 'label', 'split','left','right' (See implementation below)
    #   'label': a label node with value as the mode of the labels
    #   'split': the best split node with value as a tuple of feature id and threshold
    #   'left','right': the left and right branch with value as the label node
    # X := Tensor of size (m,d) -- Batch of m examples of demension d
    # y := Tensor(int) of size (m) -- the given vectors of labels of the examples
    # max_depth := int == the deepest level of the the decision tree
    # k := int -- the number of classes, with default value as 4
    # Return := tuple of (int, float) -- the feature id and threshold of the best split
    #

    H = entropy(y)
    if H == 0 or max_depth == 0:
        return

    best_split = find_split(node, X, y, k=k)

    ####################################
    # THIS LINE BELOW WILL REMOVE UNKNOWN OPCODE
    ####################################
    if best_split == None:
        return

    idx = X[:,best_split[0]] > best_split[1]
    X_left, y_left = X[idx], y[idx]
    X_right, y_right = X[~idx], y[~idx]

    del node['label']
    node['split'] = best_split
    node['left'] = { 'label': y_left.mode().values }
    node['right'] = { 'label': y_right.mode().values }

    ## ANSWER
    expand_node(node['left'], X_left, y_left, max_depth - 1, k)
    expand_node(node['right'], X_right, y_right, max_depth - 1, k)
    ## END ANSWER
    return

def fit_decision_tree(X,y, k=4):
    # The function will fit data with decision tree with the expand_node method implemented above

    root = { 'label': y.mode().values }
    expand_node(root, X, y, max_depth=10, k=k)
    return root

def predict_one(node, x):
    # Makes a prediction for a single example.
    # node := Map(string: value) -- the tree represented as a Map, the key will take four
    #   different string: 'label', 'split','left','right' (See implementation below)
    #   'label': a label node with value as the mode of the labels
    #   'split': the best split node with value as a tuple of feature id and threshold
    #   'left','right': the left and right branch with value as the label node
    # x := Tensor(float) of size(d,) -- the single example in a batch
    # Fill in the rest
    if 'label' in node:
        return node['label']
    feature_id, threshold = node['split']
    if x[feature_id] > threshold:
        return predict_one(node['left'], x)
    else:
        return predict_one(node['right'], x)

def predict(node, X, predict_one=predict_one):
    # return the predict result of the entire batch of examples using the predict_one function above.
    return torch.stack([predict_one(node, x) for x in X])

Test your code on the `iris` dataset. Your decision tree should fit to 100\% training accuracy and generalize to 88\% test accuracy.

In [123]:
iris = datasets.load_iris()
data=train_test_split(iris.data,iris.target,test_size=0.5,random_state=123)

X,X_te,y,y_te = [torch.from_numpy(A) for A in data]
X,X_te,y,y_te = X.float(), X_te.float(), y.long(), y_te.float()

DT = fit_decision_tree(X,y,k=4)

print('Train accuracy: ', (predict(DT, X) == y).float().mean().item())
print('Test accuracy: ', (predict(DT, X_te) == y_te).float().mean().item())

Train accuracy:  1.0
Test accuracy:  0.9200000166893005


### Bagging Decision Trees for Random forests

Note that our `find_split` implementation can use a random subset of the features when searching for the right split via the argument $k$. For the vanilla decision tree, we defaulted to $k=4$. Since there were 4 features, this meant that the decision tree could always considered all 4 features. By reducing the value of $k$ to $\sqrt(k)=2$, we can introduce variance into the decision trees for the bagging algorithm.

You'll now implement the bagging algorithm. Note that if you use the `clf` and `predict` functions given as keyword arguments, you can pass this section in the autograder without needing a correct implementation for decision trees from the previous section.

1. Bootstrap (1pt): Implement `bootstrap` to draw a random bootstrap dataset from the given dataset.
2. Fitting a random forest (1pt): Implement `random_forest_fit` to train a random forest that fits the data.
3. Predicting with a random forest (1pt): Implement `predict_forest_fit` to make predictions given a random forest.

Tip:
+ If you're not sure whether your bootstrap is working or not, remember that on average, there will be $1-1/e\approx 0.632$ unique samples in a bootstrapped dataset.

In [124]:
def bootstrap(X,y):
    # Draw a random bootstrap dataset from the given dataset.
    #
    # X := Tensor(float) of size (m,d) -- Batch of m examples of demension d
    # y := Tensor(int) of size (m) -- the given vectors of labels of the examples
    #
    # Return := Tuple of (Tensor(float) of size (m,d),Tensor(int) of size(m,)) -- the random bootstrap
    #       dataset of X and its correcting lable Y
    # Fill in the rest
    m, d = X.size()
    index = torch.randint(0, m, (m,))  # Generate random indices with replacement
    X_bs = X[index]
    y_bs = y[index]

    return X_bs, y_bs

def random_forest_fit(X, y, m, k, clf=fit_decision_tree):
    # Train a random forest that fits the data.
    # X := Tensor(float) of size (n,d) -- Batch of n examples of demension d
    # y := Tensor(int) of size (n) -- the given vectors of labels of the examples
    # m := int -- number of trees in the random forest
    # k := int -- number of classes of the features
    # clf := function -- the decision tree model that the data will be trained on
    #
    # Return := the random forest generated from the training datasets
    # Fill in the rest
    forest = []
    n, d = X.size()
    for i in range(m):
        X_bs, y_bs = bootstrap(X,y)
        tree = clf(X_bs, y_bs, k)
        forest.append(tree)

    return forest

def random_forest_predict(X, clfs, predict=predict):
    # Implement `predict_forest_fit` to make predictions given a random forest.
    # X := Tensor(float) of size (m,d) -- Batch of m examples of demension d
    # clfs := list of functions -- the random forest
    # Return := Tensor(int) of size (m,) -- the predicted label from the random forest
    # Fill in the rest
    result = []
    for clf in clfs:
        pred = predict(clf, X)
        result.append(pred)

    result = torch.stack(result)
    predicted, _ = torch.mode(result, dim=0)
    return predicted

Test your code again on the `iris` dataset. Our random forest was able to improve the accuracy of the decision tree by about 10\%!

In [125]:
torch.manual_seed(42)
RF = random_forest_fit(X,y,50,2)

print('Train accuracy: ', (random_forest_predict(X, RF) == y).float().mean().item())
print('Test accuracy: ', (random_forest_predict(X_te, RF) == y_te).float().mean().item())

Train accuracy:  1.0
Test accuracy:  0.9733333587646484


As a sanity check, the random forest can get around 95-97% test accuracy.

# 1. Boosting

In this problem, you'll implement a basic boosting algorithm on the binary classification breast cancer dataset. Here, we've provided the following weak learner for you: an $\ell_2$ regularized logistic classifier trained with gradient descent.

In [126]:
class Logistic(nn.Module):
    def __init__(self):
        super(Logistic, self).__init__()
        self.linear = nn.Linear(30,1)
    def forward(self, X):
        out = self.linear(X)
        return out.squeeze()

def fit_logistic_clf(X,y):
    clf = Logistic()
    opt = torch.optim.Adam(clf.parameters(), lr=0.1, weight_decay=1e2)
    loss = torch.nn.BCEWithLogitsLoss()
    for t in range(200):
        out = clf(X)
        opt.zero_grad()
        loss(out,(y>0).float()).backward()
        # if t % 50 == 0:
        #     print(loss(out,y.float()).item())
        opt.step()
    return clf

def predict_logistic_clf(X, clf):
    return torch.sign(clf(X)).squeeze()

Your task is to boost this logistic classifier to reduce its bias. Implement the following two functions:

+ Finish the boosting algorithm: we've provided a template for the boosting algorithm in `boosting_fit`, however it is missing several components. Fill in the missing snippets of code.
+ Prediction after boosting (2pts): implement `boosting_predict` to make predictions with a given boosted model.

In [137]:
def boosting_fit(X,y, T=10):
    # X := Tensor(float) of size (m,d) -- Batch of m examples of demension d
    # y := Tensor(int) of size (m) -- the given vectors of labels of the examples
    # T := Maximum number of models to be implemented

    m = X.size(0)
    clfs = []

    while len(clfs) < T:
        # Calculate the weights for each sample mu. You may need to
        # divide this into the base case and the inductive case.
        # mu = ...

        ## ANSWER
        if clfs == []:
          mu = torch.ones(m) / m
        else:
          mu = mu * torch.exp(alpha * misclassified)
          mu = mu / torch.sum(mu)
        ## END ANSWER

        # Here, we draw samples according to mu and fit a weak classifier
        idx = torch.multinomial(mu, m, replacement=True)
        X0, y0 = X[idx], y[idx]

        clf = fit_logistic_clf(X0, y0)

        # Calculate the epsilon error term
        # eps = ...

        ## ANSWER
        pred = predict_logistic_clf(X, clf)
        misclassified = (pred != y).float()
        eps = torch.sum(mu * misclassified)
        ## END ANSWER

        if eps > 0.5:
            # In the unlikely even that gradient descent fails to
            # find a good classifier, we'll skip this one and try again
            continue

        # Calculate the alpha term here
        # alpha = ...

        ## ANSWER
        alpha = 0.5 * torch.log2((1 - eps) / eps)
        ## END ANSWER


        clfs.append((alpha,clf))
    return clfs

def boosting_predict(X, clfs):
    # X := Tensor(float) of size (m,d) -- Batch of m examples of demension d
    # clfs := list of tuples of (float, logistic classifier) -- the list of boosted classifiers
    # Return := Tnesor(int) of size (m) -- the predicted labels of the dataset
    return torch.sign(sum(alpha*predict_logistic_clf(X, clf) for alpha,clf in clfs))


Test out your code on the breast cancer dataset. As a sanity check, your statndard logistic classifier will get a train/test accuracy of around 84%-88% while the boosted logistic classifier will get a train/test accuracy of around 90%.

In [138]:
from sklearn.datasets import load_breast_cancer
cancer = datasets.load_breast_cancer()
data=train_test_split(cancer.data,cancer.target,test_size=0.2,random_state=123)

torch.manual_seed(123)

X,X_te,y,y_te = [torch.from_numpy(A) for A in data]
X,X_te,y,y_te = X.float(), X_te.float(), torch.sign(y.long()-0.5), torch.sign(y_te.long()-0.5)


logistic_clf = fit_logistic_clf(X,y)
print("Logistic classifier accuracy:")
print('Train accuracy: ', (predict_logistic_clf(X, logistic_clf) == y).float().mean().item())
print('Test accuracy: ', (predict_logistic_clf(X_te, logistic_clf) == y_te).float().mean().item())

boosting_clfs = boosting_fit(X,y)
print("Boosted logistic classifier accuracy:")
print('Train accuracy: ', (boosting_predict(X, boosting_clfs) == y).float().mean().item())
print('Test accuracy: ', (boosting_predict(X_te, boosting_clfs) == y_te).float().mean().item())

Logistic classifier accuracy:
Train accuracy:  0.8351648449897766
Test accuracy:  0.7894737124443054
Boosted logistic classifier accuracy:
Train accuracy:  0.8703296780586243
Test accuracy:  0.9035087823867798


## Autograder

In [139]:
# Autograder will be announced on Ed Discussion approximately a week after initial release
from penngrader.grader import PennGrader

# PLEASE ENSURE YOUR PENN-ID IS ENTERED CORRECTLY. IF NOT, THE AUTOGRADER WON'T KNOW WHO
# TO ASSIGN POINTS TO YOU IN OUR BACKEND
STUDENT_ID = 72249835 # YOUR PENN-ID GOES HERE AS AN INTEGER #
SECRET = STUDENT_ID

grader = PennGrader('config.yaml', 'CIS5200_FALL_2023_HW3', STUDENT_ID, SECRET)


grader.grade(test_case_id = 'entropy', answer = entropy)
grader.grade(test_case_id = 'find_split', answer = find_split)
grader.grade(test_case_id = 'expand_node', answer = expand_node)
grader.grade(test_case_id = 'predict_one', answer = predict_one)

grader.grade(test_case_id = 'bootstrap', answer = bootstrap)
grader.grade(test_case_id = 'random_forest_fit', answer = random_forest_fit)
grader.grade(test_case_id = 'random_forest_predict', answer = random_forest_predict)

grader.grade(test_case_id = 'boosting_fit', answer = boosting_fit)
grader.grade(test_case_id = 'boosting_predict', answer = boosting_predict)

PennGrader initialized with Student ID: 72249835

Make sure this correct or we will not be able to store your grade
Correct! You earned 2/2 points. You are a star!

Your submission has been successfully recorded in the gradebook.
Correct! You earned 1/1 points. You are a star!

Your submission has been successfully recorded in the gradebook.
Correct! You earned 2/2 points. You are a star!

Your submission has been successfully recorded in the gradebook.
Correct! You earned 2/2 points. You are a star!

Your submission has been successfully recorded in the gradebook.
Correct! You earned 1/1 points. You are a star!

Your submission has been successfully recorded in the gradebook.
Correct! You earned 1/1 points. You are a star!

Your submission has been successfully recorded in the gradebook.
Correct! You earned 1/1 points. You are a star!

Your submission has been successfully recorded in the gradebook.
Correct! You earned 3/3 points. You are a star!

Your submission has been successfully